<a href="https://colab.research.google.com/github/kisaragi233/happilyeverafter2018/blob/Index-ME/SARScov2_NAIVE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Extremely naive simulation. Do NOT use as real life reference.

In [0]:
import random
from math import ceil, floor

import numpy as np

In [0]:
class people:
    def __init__(self, ID):
        self.ID = ID
        self.is_infected = False
        self.is_infectious = False
        self.is_cured = None
        self.day = 0  # int, the Xth day after infected (0 start)
        self.friendIDs = []  # list of IDs
    def _init_infection(self):  # seeding
        self.is_infected = True
        self.is_infectious = False  # patient is not immediately infectious
        self.day = 1
    def init_infection(self):
        self.is_infected = True
        self.is_infectious = False  # patient is not immediately infectious
        self.day = 0
    def prop_infection(self):  # by the end of the day
        self.day+=1
        if 4<self.day<=8:
            self.is_infectious = True
        else:
            self.is_infectious = False
    def cure(self):
        self.is_infected = False
        self.is_infectious = False
        self.is_cured = True
        
    
class population:
    def __init__(self, IDs, initial_infection_rate, rate_per_tf, 
                 mean_contacts_per_tf, std_contacts_per_tf, 
                 rate_selfCure):
        # create member instances
        self.members = {}
        self.edges = []
        for ID in IDs:
            self.members[ID] = people(ID)
        # set up community network
        # (also set up initial infection status)
        self.mean = mean_contacts_per_tf
        self.std = std_contacts_per_tf
        for idx, ID in enumerate(IDs):
            tmpTotal = np.random.normal(self.mean, self.std)
            tmpTotal = floor(max(0, min(len(IDs)-1, tmpTotal)))
            if tmpTotal==0: continue
            tmpIDs = np.random.choice([_ for _ in IDs if _!=ID], tmpTotal)
            for i in tmpIDs:
                self.members[ID].friendIDs.append(i)
                if ID not in self.members[i].friendIDs:
                    self.members[i].friendIDs.append(ID)
                edge = sorted([ID, i])
                if edge not in self.edges:
                    self.edges.append(edge)
        if initial_infection_rate==1:
            # only seed one inidividual
            ID = np.random.choice(IDs, 1)[0]
            self.members[ID].is_infected = True
            self.members[ID].is_infectious = True
            self.members[ID].day = 1
        else:
            # random seed several individuals
            assert 0<initial_infection_rate<1
            not_set = True
            for ID in IDs:
                if random.random()<initial_infection_rate:
                    self.members[ID].is_infected = True
                    not_set = False
            if not_set: self.members[np.random.choice(IDs, 1)].is_infected = True
        self.rate = rate_per_tf
        self.day = 0
        self.rateSelfCure = rate_selfCure
    def tick(self, verbose=False):
        v_counter = 0
        # propagate one timeframe
        for ID1, ID2 in self.edges:
            if self.members[ID1].is_infectious and \
                 not self.members[ID2].is_infected and \
                 not self.members[ID2].is_cured:
                if random.random()<self.rate:
                    self.members[ID2].init_infection() 
                    v_counter+=1
            elif self.members[ID2].is_infectious and \
                 not self.members[ID1].is_infected and \
                 not self.members[ID1].is_cured:
                if random.random()<self.rate:
                    self.members[ID1].init_infection()
                    v_counter+=1
        if verbose: 
            return v_counter  # number of new infection during this tick
    def prop_cycle(self, int_ticks, verbose=False):
        if verbose: print('====start of new day====')
        verbose_report = []
        # propagate infection
        # (let cured people infectionable)
        assert int_ticks>0
        v_counter_day, v_counter_tick = 0, 0
        for idx_tick in range(int_ticks):
            v_counter_tick = self.tick(verbose)
            v_counter_day+=v_counter_tick
            if verbose:
                print(' - tick {0}, new infections: {1}'.format(idx_tick, v_counter_tick))
        # propagate infection clock
        for ID in self.members:
            if self.members[ID].is_infected:
                self.members[ID].prop_infection()
        # self-cure chances by the end of the day
        for ID in self.members:
            if self.members[ID].is_infected and self.members[ID].day>6:
                if random.random()<self.rateSelfCure: 
                    self.members[ID].cure()
        # propagate wall time
        self.day+=1  # overall "wall clock"
        if verbose:
            total_infections = 0
            total_cured = 0
            for ID in self.members:
                if self.members[ID].is_infected:
                    total_infections+=1
                if self.members[ID].is_cured:
                    total_cured+=1
            print('total infections {0}, total cured {2}, new infections {1}'.format(total_infections, 
                                                                                     v_counter_day,
                                                                                     total_cured)
                 )


In [0]:
pop = population(np.arange(200), # population size (no subpopulation or anything fancy, just one blob)
                 initial_infection_rate=1,  # 1 if seeding with only 1 patient; 
                 rate_per_tf=0.001,   # the probability of infection upon *each contact* (aka one timeframe or to say one tick) with an infectious individual
                 mean_contacts_per_tf=20,  # average number of friends that one individual have (modeled simplely as a normal distribution)
                 std_contacts_per_tf=10, # standard deviation of ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                 rate_selfCure=0.7  # % chance of cure by the end of each day
                 )
for day in range(20):  # total days you want to observe
    pop.prop_cycle(10,  # how many contacts will each pair of friends make in one day
                   verbose=True
                   )